Load packages

In [69]:
#loading packages
push!(LOAD_PATH, "src");
import Base: @kwdef
using Test
using LifeInsuranceDataModel

using BitemporalPostgres
using SearchLight
using SearchLightPostgreSQL
using TimeZones
using ToStruct
using JSON

In [70]:
purging the data model entirely - empty the schema

ErrorException: syntax: extra token "the" after end of expression

In [71]:
if (haskey(ENV, "GITPOD_REPO_ROOT"))
    run(```psql -f sqlsnippets/droptables.sql```)
else
    println("execute this in Your REPL")
    println("""
        run(```sudo -u postgres psql -f sqlsnippets/droptables.sql```)
        """)
end


Loading the data model- Create tables, constraints etc. and load Roles

loading inverses of the role tables to provide role descriptions in object creation,  for instance like in: "ref_role=cpRole["Policy Holder"]

In [72]:
LifeInsuranceDataModel.load_model()

cpRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ContractPartnerRole)) do entry
    cpRole[entry.value] = entry.id.value
end
tiprRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemPartnerRole)) do entry
    tiprRole[entry.value] = entry.id.value
end
titrRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemRole)) do entry
    titrRole[entry.value] = entry.id.value
end

ppRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ProductPartRole)) do entry
    ppRole[entry.value] = entry.id.value
end


4-element Vector{Int64}:
 1
 2
 3
 4

Or just connect to an existing model

In [73]:
LifeInsuranceDataModel.connect()

PostgreSQL connection (CONNECTION_OK) with parameters:
  user = postgres
  password = ********************
  channel_binding = prefer
  dbname = postgres
  host = 127.0.0.1
  port = 5432
  client_encoding = UTF8
  options = -c DateStyle=ISO,YMD -c IntervalStyle=iso_8601 -c TimeZone=UTC
  application_name = LibPQ.jl
  sslmode = prefer
  sslcompression = 0
  sslsni = 1
  ssl_min_protocol_version = TLSv1.2
  gssencmode = prefer
  krbsrvname = postgres
  target_session_attrs = any

Create a Partner

In [74]:
p = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="Partner 1")
w = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w)
create_component!(p, pr, w)
commit_workflow!(w)

Partner1 = p.id.value

1

Create tariffs

In [75]:
# create Tariffs
function create_tariff(dsc)
    t = LifeInsuranceDataModel.Tariff()
    tr = LifeInsuranceDataModel.TariffRevision(description=dsc)
    w = Workflow(
        tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )
    create_entity!(w)
    create_component!(t, tr, w)
    commit_workflow!(w)
    t.id.value
end


LifeRiskTariff=create_tariff("Life Risk Insurance")
TerminalIllnessTariff=create_tariff("Terminal Illness")
OccupationalDisabilityTariff=create_tariff("Occupational Disability")
ProfitParticipationTariff=create_tariff("Profit participation")

find(TariffRevision)

4-element Vector{TariffRevision}:
 TariffRevision
| KEY                   | VALUE               |
|-----------------------|---------------------|
| description::String   | Life Risk Insurance |
| id::DbId              | 1                   |
| ref_component::DbId   | 1                   |
| ref_invalidfrom::DbId | 9223372036854775807 |
| ref_validfrom::DbId   | 2                   |

 TariffRevision
| KEY                   | VALUE               |
|-----------------------|---------------------|
| description::String   | Terminal Illness    |
| id::DbId              | 2                   |
| ref_component::DbId   | 2                   |
| ref_invalidfrom::DbId | 9223372036854775807 |
| ref_validfrom::DbId   | 3                   |

 TariffRevision
| KEY                   | VALUE                   |
|-----------------------|-------------------------|
| description::String   | Occupational Disability |
| id::DbId              | 3                       |
| ref_component::DbId   | 3         

In [76]:
find(Tariff,SQLWhereExpression("id=?",ProfitParticipationTariff))
find(Tariff, SQLWhereExpression("id=?", TerminalIllnessTariff))

1-element Vector{Tariff}:
 Tariff
| KEY               | VALUE |
|-------------------|-------|
| id::DbId          | 2     |
| ref_history::DbId | 3     |
| ref_version::DbId | 3     |


Create Product

In [77]:
p = Product()
pr = ProductRevision(description="Life Risk")

pp = ProductPart()
ppr = ProductPartRevision(ref_tariff=LifeRiskTariff, ref_role=ppRole["Main Coverage - Life"], description="Main Coverage - Life")

pp2 = ProductPart()
ppr2 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Lif Risk")

w0 = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w0)
create_component!(p, pr, w0)
create_subcomponent!(p, pp, ppr, w0)
create_subcomponent!(p, pp2, ppr2, w0)
commit_workflow!(w0)

LifeRiskProduct = p.id.value
println(LifeRiskProduct)

p = Product()
pr = ProductRevision(description="Life Risk - Terminal Illness")

pp = ProductPart()
ppr = ProductPartRevision(ref_tariff=LifeRiskTariff, ref_role=ppRole["Main Coverage - Life"], description="Main Coverage - Life")

pp2 = ProductPart()
ppr2 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Life Risk")

pp3 = ProductPart()
ppr3 = ProductPartRevision(ref_tariff=TerminalIllnessTariff, ref_role=ppRole["Supplementary Coverage - Terminal Illness"], description="additional cover Terminal Illness")

pp4 = ProductPart()
ppr4 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Terminal Illness")

pp5 = ProductPart()
ppr5 = ProductPartRevision(ref_tariff=OccupationalDisabilityTariff, ref_role=ppRole["Supplementary Coverage - Occupational Disablity"], description="additional cover Occupational Disablity")

pp6 = ProductPart()
ppr6 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Occ.Disablity")



w0 = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w0)
create_component!(p, pr, w0)
create_subcomponent!(p, pp, ppr, w0)
create_subcomponent!(p, pp2, ppr2, w0)
create_subcomponent!(p, pp3, ppr3, w0)
create_subcomponent!(p, pp4, ppr4, w0)
create_subcomponent!(p, pp5, ppr5, w0)
create_subcomponent!(p, pp6, ppr6, w0)
commit_workflow!(w0)

LifeRiskTIODProduct = p.id.value
println(LifeRiskTIODProduct)

Create contract

In [78]:
    w1 = Workflow(
        tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )
    
    create_entity!(w1)
    c = Contract()
    cr = ContractRevision(description="contract creation properties")
    create_component!(c, cr, w1)
    
    cpr = ContractPartnerRef(ref_super=c.id)
    cprr = ContractPartnerRefRevision(ref_partner=Partner1, ref_role=cpRole["Policy Holder"], description="policiyholder ref properties")
    create_subcomponent!(c, cpr, cprr, w1)
    # pi 1
    cpi = ProductItem(ref_super=c.id)
    cpir = ProductItemRevision(position=1, ref_product=LifeRiskTIODProduct, description="from contract creation")
    create_subcomponent!(c, cpi, cpir, w1)
    # pi 1 ti 1 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=LifeRiskTariff, ref_role=titrRole["Main Coverage - Life"], description="Life Risk tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 1 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 2 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Profit participation"], description="Profit participation tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 2 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 3 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=TerminalIllnessTariff, ref_role=titrRole["Supplementary Coverage - Terminal Illness"], description="Terminal Illness tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 3 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 4 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Profit participation"], description="Profitparticipation tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 4 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    
    # pi 1 ti 5 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Supplementary Coverage - Occupational Disablity"], description="Occupational Disablity tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 5 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 6 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Profit participation"], description="Profit Participation tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 6 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    
    
    
    commit_workflow!(w1)

UndefVarError: UndefVarError: o not defined

update Contract yellow

In [79]:
cr1 = ContractRevision(ref_component=c.id, description="contract 1, 2nd mutation")
w2 = Workflow(
    ref_history=w1.ref_history,
    tsw_validfrom=ZonedDateTime(2016, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
update_entity!(w2)
update_component!(cr, cr1, w2)
commit_workflow!(w2)

   update Contract red

In [80]:

cr2 = ContractRevision(ref_component=c.id, description="contract 1, 3rd mutation retrospective")
w3 = Workflow(
    ref_history=w2.ref_history,
    tsw_validfrom=ZonedDateTime(2015, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
update_entity!(w3)
update_component!(cr1, cr2, w3)
commit_workflow!(w3)
